In [ ]:
# инсталлируем библиотеку для работы с pdf

!pip install pdfminer.six

In [ ]:
# импортируем нужные модули

import pdfminer.high_level
import requests
import os
import re
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import numpy as np

##Функция №1
Эта функция, получает на вход URL-ссылку на PDF файл и переводит его в текст.

In [ ]:
'''
Эта функция принимает на вход URL-адрес
по которому находится pdf файл, переводит его в текстовый формат
и сохраняет на диск.

  Вход:
    url - адрес PDF файла
  Выход:
    topic - текст
'''

def pdf_to_txt(url):
  topic = ''
  pattern = re.compile(r'\w[а-я]')
  # получаем файл
  pdf_req = requests.get(url)
  # проверяем, что страница удачно парсится
  if pdf_req.status_code == 200:
    # запоминаем название файла без расширения
    f_name = os.path.basename(url).split('.')[0]
    # записываем файл на диск
    with open(f'{f_name}.pdf','wb') as f:
      f.write(pdf_req.content)
    # переводим файл в текст
    text = pdfminer.high_level.extract_text(f'{f_name}.pdf')
    # удаляем pdf файл
    if os.path.isfile(f'{f_name}.pdf'):
      os.remove(f'{f_name}.pdf')
    # исправляем некоторые ошибки парсинга
    text = text.replace(' \n', ' ')
    text = text.replace('-\n', '-')
    # разбиваем текст на строки для дальнейшей обработки
    txt_split = text.split('\n')
    # убираем пробелы в начале и конце каждой строки
    for st in range(len(txt_split)):
      res = txt_split[st].strip()
      # убираем повторяющиеся пробелы внутри строки
      res = re.sub('\s+',' ', res)
      # если строка не пустая и не номер страницы - добавляем её к тексту
      if res and not res.isdigit() and (len(res)>1):
        # это условие убирает ошибочные разрывы абзаца
        if re.match(r'[а-я]', res[0]):
          topic = topic[:-1]
          topic += ' '
        topic += res+'\n'
    # записываем текст в файл с названием оригинала, но txt
    with open(f'{f_name}.txt','w') as f:
      f.write(topic)

    return topic


In [ ]:
# пррверяем как работает функция на конкретном файле

url = 'https://cdn.kia.ru/master-data/brochures/Kia_Corporate_Sales.pdf'

print(pdf_to_txt(url))

##Функция №2
Эта функция, получает на вход URL-ссылку на страницу сайта и выбирает из неё все ссылки на PDF файлы. Возвращает в виде словаря.

In [ ]:
def find_pdf(url):
  # Получаем запрос от страницы
  response = requests.get(url)
  # Получаем ПрекрасныйСуп
  bs = BeautifulSoup(response.text, "html.parser")

  # Определяем функцию отбора тегов. Берём теги у которых есть ссылка.
  def is_href(href):
    return href

  # Отбитаем теги у которых есть ссылка
  bs.find_all(href = is_href)

  # Заводим пустой словарь
  PDF_name = {}

  # Наполняем словарь описаниями файла и ссылками
  # То, что это словарь - важно. Так мы избегаем одинаковых файлов, но с разными ссылками. (такое есть)
  for tag in bs.find_all(href = is_href):
      # Извлекаем ссылку из тега
      tag_href = tag.attrs['href']
      # Делаем условие, если ссылка на файл, который оканчивается на .pdf
      if tag_href.split('.')[-1] == 'pdf':
        # Добавляем новый элемент в словарь
        PDF_name[tag.get_text().strip()] = tag_href
  # Возвращаем словарь
  return PDF_name

##Смотрим как работает функция на примере одного URL.

In [ ]:
# Смотрим как работает функция

# Берём произвольную страницу из БЗ
url = 'https://www.kia.ru/auction/'

# Делаем запрос и извлекаем название страницы
response = requests.get(url)
bs = BeautifulSoup(response.text, "html.parser")

# Извлекаем "словарь" (список PDFок с сайта)
PDFs = find_pdf(url)

# Условие: Что-то печаем, если PDF на странице сайта есть
if PDFs:
  # Если PDFки есть, то то печатаем заголовок
  print(f'Страница: {bs.title.string}')
  print(f'Адрес страницы: {url}')

  # Распечатываем PDF файлы, которые есть на этой странице
  for key,value in PDFs.items():
    print(f'PDF - "{key}"\n\tссылка: {value}')


##Функция №3
А теперь проходимся по всей базе и находим все PDFки.
Сначала подгружаем БД:


Поскольку я начал в парсинге заниматься PDF, то могу сказать, что в PDF информации даже чуть больше, чем на страницах.
Цикл проходтися по всем нашим ссылкам и выдирает все PDFки.

In [ ]:
# cкачиваем фалы данных в виде csv

def load_document_csv(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=tsv')
    response.raise_for_status()
    text = response.content

    return text

urls = load_document_csv('https://docs.google.com/spreadsheets/d/1h5BNBTXwzb8nMTIcre0vCDcujax1Fvfw3RdwT7FeAt8/edit?usp=sharing')
with open('urls_r.csv','wb') as f:
  f.write(urls)

In [ ]:
# Дальше загружаем из файла данных в массим Numpy

links = np.genfromtxt('urls_r.csv', delimiter='\t', dtype=str, encoding='utf-8' )

In [ ]:
# Идём по списку ссылок
for row in links:

  # Делаем запрос и извлекаем URL стреницы, который находится в столбце [1]
  response = requests.get(row[1])
  bs = BeautifulSoup(response.text, "html.parser")

  # Ищем все вхождения PDF (описание функции выше)
  PDFs = find_pdf(row[1])

  # Условие: Что-то печатаем, если PDF на странице сайта есть
  if PDFs:
    # Если PDFки есть, то печатаем заголовок
    print(f'\nСтраница: {bs.title.string}')
    print(f'Адрес страницы: {row[1]}')

    # Распечатываем PDF файлы, которые встречаются на страницах
    for key,value in PDFs.items():
      print(f'PDF - "{key}"\n\tссылка: {value}')


И теперь мы можем преобразовать все PDF файлы в текст при помощи Функции №1. Разделить его на чанки любым из доступных способов, а также придумать свой, оригинальный. ))